# Preprocessing

In [6]:
import nltk
raw1 = 'Hello,Healco,Heato,Ahelt,this is a test corpus:Corpus linguistics proposes that reliable language analysis is more feasible with corpora collected in the field, in their natural contexts, and with minimal experimental-interference.'
tokens = nltk.word_tokenize(raw1)
print(tokens)
## change tokens to lowercase
tokens=[word.lower() for word in tokens if word.isalpha()]
print(tokens)

['Hello', ',', 'Healco', ',', 'Heato', ',', 'Ahelt', ',', 'this', 'is', 'a', 'test', 'corpus', ':', 'Corpus', 'linguistics', 'proposes', 'that', 'reliable', 'language', 'analysis', 'is', 'more', 'feasible', 'with', 'corpora', 'collected', 'in', 'the', 'field', ',', 'in', 'their', 'natural', 'contexts', ',', 'and', 'with', 'minimal', 'experimental-interference', '.']
['hello', 'healco', 'heato', 'ahelt', 'this', 'is', 'a', 'test', 'corpus', 'corpus', 'linguistics', 'proposes', 'that', 'reliable', 'language', 'analysis', 'is', 'more', 'feasible', 'with', 'corpora', 'collected', 'in', 'the', 'field', 'in', 'their', 'natural', 'contexts', 'and', 'with', 'minimal']


# Permuterm index
Now we define the method to generate Permuterm index

In [2]:
def Generate_Permuterm_index(raw):
    Permuterm_index = []
    for token in raw:
        index = ''
        cursor = 0
        index = '$' + token
        Permuterm_index.append(index)
        while cursor <= len(token):
            index = ''
            if cursor == 0:
                cursor += 1
                continue
            else:
                index = token[-cursor:] + '$' + token[:-cursor]
            Permuterm_index.append(index)            
            cursor += 1        
    return Permuterm_index        

Test the Generate_Permuterm_index method using a corpus

In [3]:
permuterm = Generate_Permuterm_index(tokens)
print(Generate_Permuterm_index(tokens))

['$hello', 'o$hell', 'lo$hel', 'llo$he', 'ello$h', 'hello$', '$healco', 'o$healc', 'co$heal', 'lco$hea', 'alco$he', 'ealco$h', 'healco$', '$heato', 'o$heat', 'to$hea', 'ato$he', 'eato$h', 'heato$', '$ahelt', 't$ahel', 'lt$ahe', 'elt$ah', 'helt$a', 'ahelt$', '$this', 's$thi', 'is$th', 'his$t', 'this$', '$is', 's$i', 'is$', '$a', 'a$', '$test', 't$tes', 'st$te', 'est$t', 'test$', '$corpus', 's$corpu', 'us$corp', 'pus$cor', 'rpus$co', 'orpus$c', 'corpus$', '$corpus', 's$corpu', 'us$corp', 'pus$cor', 'rpus$co', 'orpus$c', 'corpus$', '$linguistics', 's$linguistic', 'cs$linguisti', 'ics$linguist', 'tics$linguis', 'stics$lingui', 'istics$lingu', 'uistics$ling', 'guistics$lin', 'nguistics$li', 'inguistics$l', 'linguistics$', '$proposes', 's$propose', 'es$propos', 'ses$propo', 'oses$prop', 'poses$pro', 'oposes$pr', 'roposes$p', 'proposes$', '$that', 't$tha', 'at$th', 'hat$t', 'that$', '$reliable', 'e$reliabl', 'le$reliab', 'ble$relia', 'able$reli', 'iable$rel', 'liable$re', 'eliable$r', 'reliab

# The query processing algorithm
The query processing algorithm takes a query containing * and return the answer. Note that there could be more than 1 * in the query.

In [4]:
def wildcard_query(query,permuterm_index):
    count = 0
    ## calculate tha amount of wildcard in query so that we can consider separately
    for symbol in query:
        if symbol == '*':
            count += 1
    result = []
    ## if the amount of wildcard equals to 1
    if count == 1:
        splited_query = query.split('*')
        if query[-1] == '*':        
            for term in permuterm_index:
                pattern = term
                splited_parts = pattern.split('$')
                if splited_parts[1] == splited_query[0]:                
                    result.append(splited_parts[1] + splited_parts[0])
        elif query[0] == '*':
            for term in permuterm_index:
                pattern = term
                splited_parts = pattern.split('$')
                if splited_parts[0] == splited_query[1]:
                    result.append(splited_parts[1] + splited_parts[0])
        elif query[0] != '*' and query [-1] != '*':
            suffix_length = len(splited_query[0])
            for term in permuterm_index:
                pattern = term
                splited_parts = pattern.split('$')
                if suffix_length > len(splited_parts[1]):
                    continue
                suffix = splited_parts[1][:suffix_length]
                if splited_parts[0] == splited_query[1] and splited_query[0] == suffix:
                    result.append(splited_parts[1] + splited_parts[0])
    ## if the amount of wildcard is larger than 1
    elif count > 1:
        temp_result = []
        splited_query = query.split('*')
        suffix_length = len(splited_query[0])
        length = len(splited_query[2])
        ## the wildcards locates in the middle of query word
        if query[0] != '*' and query [-1] != '*':
            for term in permuterm_index:
                pattern = term
                splited_parts = pattern.split('$')
                if suffix_length > len(splited_parts[1]):
                    continue
                suffix = splited_parts[1][:suffix_length]
                if splited_parts[0] == splited_query[2] and splited_query[0] == suffix:
                    temp_result.append(splited_parts[1] + splited_parts[0])
            for term in temp_result:
                if splited_query[1] in term[suffix_length:-length]:
                    result.append(term)
        ## the heading and trailing characters are wildcard
        elif query[0] == '*' and query [-1] == '*': 
            suffix_length = len(splited_query[1])
            for term in permuterm_index:
                pattern = term
                splited_parts = pattern.split('$')
                if splited_query[1] in splited_parts[1]:                
                    result.append(splited_parts[1] + splited_parts[0])
            result = list(set(result))
    return result

Test the wildcard query(Notice we support the query containing 2 wildcard at most). Our corpus is 'Hello,Healco,Heato,Ahelt,this is a test corpus:Corpus linguistics proposes that reliable language analysis is more feasible with corpora collected in the field, in their natural contexts, and with minimal experimental-interference.'

In [5]:
print(wildcard_query('cor*',permuterm))
print(wildcard_query('*al',permuterm))
print(wildcard_query('hel*o',permuterm))
print(wildcard_query('he*l*o',permuterm))
print(wildcard_query('*hel*',permuterm))

['corpus', 'corpus', 'corpora']
['natural', 'minimal']
['hello']
['hello', 'healco']
['ahelt', 'hello']
